In [45]:
import numpy as np
from scipy.optimize import linprog
import pandas as pd

In [15]:
def NN2NA(NN):
  rows,columns=NN.shape

  columns_NA=np.count_nonzero(NN)
  if(rows != columns):
    print("Malformed NN Matrix. rows are not equal to columns")
    return none,none
  k=0
  NA=np.zeros((rows,columns_NA))
  arches = ["" for i in range(columns_NA)]

  for i in range(columns):
    for j in range(rows):
      if NN[i,j]==1:
        NA[i,k]=1
        NA[j,k]=-1
        #if i==0:
        #  arches[k]="s->"+str(j+1)
        #elif j==0:
        #  arches[k]=str(i+1)+"->s"
        #elif i==columns-1:
        #  arches[k]="t->"+str(j+1)
        #elif j==columns-1:
        #  arches[k]=str(i+1)+"->t"
        #else:
        #  arches[k]=str(i+1)+"->"+str(j+1)
        arches[k]=str(i+1)+"->"+str(j+1)
        k+=1
  return NA, arches


def get_usage_string(arc_idxs, res_flow, capacity):
    return {arc: '%s/%s' % (flow, cap) for arc, flow, cap in zip(arc_idxs, res_flow, capacity)}

def get_min_cut(arc_idxs, np_res_flow, np_capacity):
    np_capacity = np.where(np_capacity == None, 999, np_capacity)

    idxs = np.argwhere((np_res_flow - np_capacity) == 0)
    return [arc_idxs[i[0]] for i in idxs]

In [39]:
               #P1A P1B P2A P2B P3A P3B S1A S1B S2A S2B V1A V1B V2A V3B V3A V3B
NN = np.array([
               [0,  0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,  0,  0],   #P1A
               [0,  0,  0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,  0],   #P1B
               [0,  0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,  0,  0],   #P2A
               [0,  0,  0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,  0],   #P2B
               [0,  0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,  0,  0],   #P3A
               [0,  0,  0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,  0],   #P3B
               [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  1,  0,  1,  0],   #S1A
               [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  1,  0,  1],   #S1B
               [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  1,  0,  1,  0],   #S2A
               [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  1,  0,  1],   #S2B
               [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],   #V1A
               [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],   #V1B
               [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],   #V2A
               [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],   #V2B
               [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],   #V3A
               [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])  #V3B

C = np.array([100, 100, 200, 200, 150, 150, 150, 150, 200, 200, 100, 100,
              100, 150, 200, 200, 150, 100, 100, 150, 200, 200, 150, 100])



NA, arc_idxs = NN2NA(NN)

p=6
s=4
v=6

#Arrays for first restriction
MASK=[]
a = [np.ones(p*NA.shape[1])]
MASK=np.append(MASK,a)
a = [np.zeros((s+v)*NA.shape[1])]
MASK=np.append(MASK,a)
MASK=MASK.reshape((p+s+v),NA.shape[1])
Aub=(np.multiply(NA,MASK))

bub = np.array([20, 30, 10, 40, 30, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
bub=np.transpose(bub)

#Arrays for second restriction
MASK=[]
a = [np.zeros(p*NA.shape[1])]
MASK=np.append(MASK,a)
a = [np.ones((s+v)*NA.shape[1])]
MASK=np.append(MASK,a)
MASK=MASK.reshape((p+s+v),NA.shape[1])
Aeq=(np.multiply(NA,MASK))


beq = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -30, -40, -10, -20, -20, -20]) 
beq = np.transpose(bm)

max_q=None

bounds = tuple([(0, max_q) for arcs in range(0, Aeq.shape[1])])

print('## Optimizer inputs ## \n'
      'Cost vector: %s \n\n '
      'Arches in order:\n%s \n\n'
      'A_eq Node-Arc matrix:\n%s \n\n'
      'b_eq demand-supply vector: %s \n\n'
      'Bounds of each X arc variable: %s' % (C,arc_idxs,Aeq, beq, bounds))

## Optimizer inputs ## 
Cost vector: [100 100 200 200 150 150 150 150 200 200 100 100 100 150 200 200 150 100
 100 150 200 200 150 100] 

 Arches in order:
['1->7', '1->9', '2->8', '2->10', '3->7', '3->9', '4->8', '4->10', '5->7', '5->9', '6->8', '6->10', '7->11', '7->13', '7->15', '8->12', '8->14', '8->16', '9->11', '9->13', '9->15', '10->12', '10->14', '10->16'] 

A_eq Node-Arc matrix:
[[0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
  0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]
 [0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
  0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]
 [0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
  0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]
 [0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
  0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]
 [0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
  0.00 0.00 0.00 0.00 0.00 0.00 0.00 

In [58]:
# OPTIMIZE:

res = linprog(C, A_eq=Aeq, b_eq=beq, A_ub=Aub, b_ub=bub, bounds=bounds, method='simplex')

# GET THE SOLUTION:
#usage = get_usage_string(arc_idxs, res.x.astype(int), max_q)
min_cut = get_min_cut(arc_idxs, res.x, np.array(max_q))
max_flow = res.fun * -1

np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})
print('## Results ##')
#print('The usage of each arc will be %s' % usage)
print('The arcs that produce the minimum cut : %s' % min_cut)
print('The maximum flow will be: %0.2f\n\n\n' % max_flow)

## Results ##
The arcs that produce the minimum cut : []
The maximum flow will be: -44000.00





In [61]:
print('Results:\n' % max_flow)
df=pd.DataFrame(res.x,index=arc_idxs,columns=['Quantity'])
print(df)

Results:

        Quantity
1->7        20.0
1->9         0.0
2->8        30.0
2->10        0.0
3->7        10.0
3->9         0.0
4->8        40.0
4->10        0.0
5->7        30.0
5->9         0.0
6->8        10.0
6->10        0.0
7->11       30.0
7->13       10.0
7->15       20.0
8->12       40.0
8->14       20.0
8->16       20.0
9->11        0.0
9->13        0.0
9->15        0.0
10->12       0.0
10->14       0.0
10->16       0.0
